### Use the menu entry "Run &rarr; Run All Cells" to start this notebook.
This interactive Jupyter notebook is supported by <a href="https://www.mozilla.org/firefox/new/" target="_blank">Firefox</a> and
<a href="https://en.wikipedia.org/wiki/Chromium_(web_browser)#Browsers_based_on_Chromium" target="_blank">Chromium-based browsers</a> like Chrome, Opera, Vivaldi, Brave, Edge and others.

In [ ]:
# main program code

from datetime import datetime
start_time = datetime.now()

# phase 1: install all necessary packages
import gettext
import piplite
from IPython.display import clear_output, display, HTML

gettext.bindtextdomain('kuba', 'translations')
gettext.textdomain('kuba')
_ = gettext.gettext

text_template = _('Installing package {package}')

# install ipywidgets to be able to show a real progress bar
print(text_template.format(package="ipywidgets"))
await piplite.install('ipywidgets==8.1.3')

clear_output()

packages = [
    'babel',
    'folium',
    'geopandas',
    'itables',
    'ipyleaflet',
    'mapclassify',
    'openpyxl',
    'pandas',
    'plotly'
]

# there will be 6 additional steps later in the KUBA constructor
from ProgressBar import ProgressBar
progress_bar = ProgressBar(len(packages) + 6)

for package in packages:
    installing_text = text_template.format(package=package)
    progress_bar.update_progress(description=installing_text)
    await piplite.install(package)

# phase 2: all necessary packages are installed
# (start the real program)
import ipywidgets as widgets

try:
    from KUBA import KUBA

    kuba = KUBA(progress_bar)

    def loadButtonClicked(b):
        kuba.loadBridges()

    def togglePocMarkersLayer(b):
        kuba.bridges_poc_map.toggle_marker_layers()

    def toggleRiskMarkersLayer(b):
        kuba.bridges_risk_map.toggle_marker_layers()

    def updateReadout(b):
        kuba.updateReadout()

    kuba.loadButton.on_click(loadButtonClicked)
    kuba.bridges_poc_map.cluster_button.observe(togglePocMarkersLayer, names="value")
    kuba.bridges_risk_map.cluster_button.observe(toggleRiskMarkersLayer, names="value")
    kuba.bridgesSlider.observe(updateReadout, names="value")

    end_time = datetime.now()
    elapsed_time = (end_time - start_time).total_seconds()
    print(_("Program started at: {start_time}").format(start_time=start_time))
    print(_("Program completed at: {end_time}").format(end_time=end_time))
    print(_("Elapsed time: {elapsed_time} seconds").format(elapsed_time=elapsed_time))

except ModuleNotFoundError:
    display(widgets.HTML(_(
                """
                <h1>Startup failed</h1>
                The startup of this notebook has failed. A known cause for this
                error is starting the notebook in Firefox in private mode.
                Please try again in a new Firefox window in normal mode. More
                background information about this problem can be found here:
                <br>
                <a href="https://jupyterlite.readthedocs.io/en/latest/howto/configure/advanced/service-worker.html" target="_blank">
                https://jupyterlite.readthedocs.io/en/latest/howto/configure/advanced/service-worker.html</a>
                """
            )))